In [3]:
import pandas as pd
import numpy as np
import csv

In [5]:
result_file_path = "/Users/kiyoshi/Desktop/pred_results_369_44_0731.csv"

In [10]:
result_pd = pd.read_csv(result_file_path, names=["ground_truth", "pred", "RT"])

In [11]:
result_pd.head(10)

,ground_truth,pred,RT
0,0,60,0.907778
1,0,0,0.477815
2,0,0,0.329069
3,0,0,0.254605
4,0,0,0.210552
5,0,0,0.180778
6,0,0,0.159433
7,0,0,0.143421
8,0,0,0.130962
9,0,0,0.121223


In [12]:
len(result_pd)

43850

In [13]:
# Get the over all data for known and unknown first
known_pd = result_pd[result_pd["ground_truth"] != -1]
unknown_pd = result_pd[result_pd["ground_truth"] == -1]

In [15]:
print(len(known_pd))
print(len(unknown_pd))

39450
4400


In [17]:
nb_known_pred_known = len(known_pd[known_pd["pred"] != -1])
nb_known_pred_unknown = len(known_pd[known_pd["pred"] == -1])

nb_unknown_pred_unknown = len(unknown_pd[unknown_pd["pred"] == -1])
nb_unknown_pred_known = len(unknown_pd[unknown_pd["pred"] != -1])

In [18]:
print(nb_known_pred_known, 
      nb_known_pred_unknown,
     nb_unknown_pred_known,
     nb_unknown_pred_unknown)

35672 3778 3642 758


In [19]:
known_pd["RT"].mean()

0.034617902586026046

In [20]:
unknown_pd["RT"].mean()

0.03449683273315459

In [21]:
known_pd[known_pd["pred"] != -1]["RT"].mean()

0.03463020645869208

In [22]:
known_pd[known_pd["pred"] == -1]["RT"].mean()

0.034501729016480985

In [23]:
unknown_pd[unknown_pd["pred"] == -1]["RT"].mean()

0.034495786890913885

In [24]:
unknown_pd[unknown_pd["pred"] != -1]["RT"].mean()

0.03449705040158371